In [1]:
import sidekit
import os
import sys
import multiprocessing
import matplotlib.pyplot as mpl
import logging
import re
import numpy as np
import itertools

In [2]:
distribNb = 512
rsr2015Path = './'

# Default for RSR2015
audioDir = os.path.join(rsr2015Path , 'data')

# Automatically set the number of parallel process to run.
# The number of threads to run is set equal to the number of cores available
# on the machine minus one or to 1 if the machine has a single core.
nbThread = max(multiprocessing.cpu_count()-1, 1)

In [48]:
import itertools
"""
GMMのファイルパス登録
"""
gmm_wav_path = []
mic = ["WAVES_DT","WAVES_HS"]
per = os.listdir(audioDir+"/WAVES_DT")
for k in ["/".join([audioDir,i,j,"PB"]) for i,j in itertools.product(mic,per)]:
    #print(k[7:])
    if os.listdir(k):
        for l in os.listdir(k):
            gmm_wav_path.append(k[7:]+"/"+l[:-4])
ubmList = gmm_wav_path

In [64]:
"""
ファイルの確認
"""
for i in ubmList:
    if os.path.exists(audioDir+"/"+i+".wav"):
        a = "a"
    else:
        print("ねえ！")

In [3]:
print('Load task definition')
enroll_idmap = sidekit.IdMap('task/3sesspwd_eval_m_trn.h5')
test_ndx = sidekit.Ndx('task/3sess-pwd_eval_m_ndx.h5')
key = sidekit.Key('task/3sess-pwd_eval_m_key.h5')
#with open('task/ubm_list.txt') as inputFile:
    #ubmList = inputFile.read().split('\n')

Load task definition


In [4]:
print(enroll_idmap)

------------------------------
left ids:array(['1', '1', '1', ..., '100', '100', '100'], dtype='<U255')
right ids:array(['1/1', '1/2', '1/3', ..., '100/98', '100/99', '100/100'],
      dtype='<U255')
seg start:array([None, None, None, ..., None, None, None], dtype=object)
seg stop:array([None, None, None, ..., None, None, None], dtype=object)
------------------------------



In [4]:
logging.info("Initialize FeaturesExtractor")
extractor = sidekit.FeaturesExtractor(audio_filename_structure=audioDir+"/{}.wav",
                                      feature_filename_structure="./features/{}.h5",
                                      sampling_frequency=16000,
                                      lower_frequency=133.3333,
                                      higher_frequency=6955.4976,
                                      filter_bank="log",
                                      filter_bank_size=40,
                                      window_size=0.025,
                                      shift=0.01,
                                      ceps_number=19,
                                      vad="snr",
                                      snr=40,
                                      pre_emphasis=0.97,
                                      save_param=["vad", "energy", "cep"],
                                      keep_all_features=False)


In [5]:
# Create a FeaturesServer to load features and feed the other methods
features_server = sidekit.FeaturesServer(features_extractor=None,
                                         feature_filename_structure="./features/{}.h5",
                                         sources=None,
                                         dataset_list=["energy", "cep", "vad"],
                                         mask=None,
                                         feat_norm="cmvn",
                                         global_cmvn=None,
                                         dct_pca=False,
                                         dct_pca_config=None,
                                         sdc=False,
                                         sdc_config=None,
                                         delta=True,
                                         double_delta=True,
                                         delta_filter=None,
                                         context=None,
                                         traps_dct_nb=None,
                                         rasta=True,
                                         keep_all_features=False)

In [6]:
ubm = sidekit.Mixture("./gmm/ubm.h5")

In [7]:
enroll_sv = sidekit.StatServer('data/sv_enroll.h5')

In [8]:
scores_gmm_ubm =  sidekit.bosaris.Scores('scores/scores_gmm-ubm.h5')

In [43]:
sv = enroll_sv.stat1
sv.shape

(100, 30720)

In [93]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(sv.T)
ev = pca.components_.T #固有声
ev.shape

(100, 5)

In [101]:
import pandas as pd
import scipy
df = pd.DataFrame(ev)
df_zs = df.apply(scipy.stats.zscore, axis=0)
#df.to_excel('evc.xlsx')
with pd.ExcelWriter('evc.xlsx') as writer:
    df.to_excel(writer, sheet_name='sheet1')
    df_zs.to_excel(writer, sheet_name='sheet2')

In [86]:
pca.explained_variance_ratio_

array([0.89168442, 0.01120106, 0.0068345 , 0.00559644, 0.00441676])

In [95]:
ev_zs = scipy.stats.zscore(ev)

In [97]:
data = []
for x,j in enumerate(ev_zs):
    c = 0
    for i in range(5):
        if j[i] > 0:
            c += 2**(4-i)
    data.append([x+1,c])
list = sorted(data, key=lambda x: x[1])
for i in list:
    print(i)

[46, 0]
[87, 0]
[100, 0]
[5, 1]
[17, 1]
[22, 1]
[42, 1]
[98, 1]
[4, 2]
[51, 2]
[49, 3]
[15, 4]
[38, 4]
[90, 4]
[14, 5]
[69, 5]
[72, 5]
[6, 6]
[40, 6]
[45, 6]
[57, 6]
[86, 6]
[88, 6]
[99, 6]
[1, 7]
[44, 7]
[89, 8]
[53, 10]
[97, 10]
[18, 11]
[35, 12]
[55, 12]
[84, 12]
[68, 14]
[71, 14]
[93, 14]
[2, 15]
[7, 15]
[10, 15]
[11, 15]
[52, 15]
[65, 15]
[3, 17]
[16, 17]
[73, 18]
[26, 21]
[27, 21]
[28, 21]
[29, 21]
[30, 21]
[31, 21]
[43, 21]
[47, 21]
[70, 21]
[74, 21]
[75, 21]
[76, 21]
[77, 21]
[78, 21]
[80, 21]
[81, 21]
[91, 21]
[41, 22]
[66, 22]
[24, 23]
[32, 23]
[9, 24]
[21, 24]
[23, 24]
[37, 24]
[39, 24]
[62, 24]
[63, 25]
[83, 25]
[56, 26]
[92, 26]
[12, 27]
[13, 27]
[19, 27]
[34, 27]
[36, 27]
[50, 27]
[54, 27]
[59, 27]
[67, 27]
[79, 27]
[33, 28]
[60, 28]
[61, 28]
[82, 28]
[85, 28]
[94, 28]
[96, 28]
[8, 29]
[20, 29]
[25, 29]
[64, 29]
[95, 29]
[58, 30]
[48, 31]


In [100]:
import shutil
for i in range(32):
    if not os.path.exists("./result/{}".format(i)):
        os.mkdir("./result/{}".format(i))
for i in list:
    x = "./data/{}/1.wav".format(i[0])
    y = "./result/{}/{}.wav".format(i[1],i[0])
    shutil.copyfile(x,y)

In [102]:
prior = sidekit.logit_effective_prior(0.01, 1, 1)
dp = sidekit.DetPlot(window_style='sre10',
                    plot_title="GMM-UBM")
dp.set_system_from_scores(scores_gmm_ubm,
                         key, sys_name="GMM-UBM")
minDCF, Pmiss, Pfa, prebep, EER = sidekit.bosaris.detplot.fast_minDCF(
dp.__tar__[0], dp.__non__[0], prior, normalize=True)
print("UBM-GMM , minDCF = {}, eer = {}".format(minDCF,EER))

UBM-GMM , minDCF = 0.17775332599779933, eer = 0.10944900945673351
